We want to calculate the covariance matrix of our background data.



Fun fact: our covariance array is a Toeplitz matrix

In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
full_time_series = torch.from_numpy(np.loadtxt("./data/background_0.dat", dtype=np.float32))
full_time_series = full_time_series.to(device)

time_series = full_time_series

In [3]:
try:
    torch.cuda.reset_peak_memory_stats()
except:
    print("Cuda not supported or something like that")

cov_mat_row_one = []
means = []

sliding_window_size = 1000
batch_size = 100 # experimental so it runs well, must divide sliding_window_size evenly

y0_samples = time_series.unfold(0, sliding_window_size, 1).T[0]
y0_diff = y0_samples - torch.mean(y0_samples)

for i in range(int(sliding_window_size / batch_size)):
    samples = time_series.unfold(0, sliding_window_size, 1).T[i * batch_size:(i + 1) * batch_size]
    mean = torch.mean(samples, dim=1)
    diff = (samples.T - mean).T
    cov_mat_row_one.append(diff @ y0_diff / y0_samples.shape[0])
    means.append(mean)
    del samples
    del mean
    del diff

# clear memory
with torch.no_grad(): # not sure if this line is correct
    torch.cuda.empty_cache()
    
cov_mat_row_one = torch.flatten(torch.stack(cov_mat_row_one))
means = torch.flatten(torch.stack(means))

print("Current GPU MB allocated: " + str(torch.cuda.memory_allocated() / 1024 / 1024))
print("Max GPU MB allocated: " + str(torch.cuda.max_memory_allocated() / 1024 / 1024))

Cuda not supported or something like that
Current GPU MB allocated: 0.0
Max GPU MB allocated: 0.0


In [4]:
try:
    torch.cuda.reset_peak_memory_stats()
except:
    print("Cuda not supported or something like that")

cov_mat = torch.Tensor(scipy.linalg.toeplitz(cov_mat_row_one.cpu().numpy())).to(device)
inv_cov_mat = torch.inverse(cov_mat)

print("Current GPU MB allocated: " + str(torch.cuda.memory_allocated() / 1024 / 1024))
print("Max GPU MB allocated: " + str(torch.cuda.max_memory_allocated() / 1024 / 1024))

Cuda not supported or something like that
Current GPU MB allocated: 0.0
Max GPU MB allocated: 0.0


In [7]:
torch.save(cov_mat, "saved_tensors/cov_mat_background_0_1000.pt")
torch.save(inv_cov_mat,"saved_tensors/inv_cov_mat_background_0_1000.pt")